# Decision Trees (Heart Disease Prediction First Status Report)
## Alright, time for another stab at heart disease prediction with decision trees!
Last time, I redid some of the data cleaning work, performed feature selection on the dataset, performed a grid search for optimal tree depth,and made predictions with a depth 6 decision tree.  
### The test error rate from the last tree was 0.265. This is my target to beat this time around.
I have a couple improvements in mind. First of all, the data set cleaning has been redone as a team, so I won't have to redo it. Second, I want to perform feature selection with decision trees, rather than logistic regressors, and in the process correct some errors I made with feature selection last time. Third, I want to run a grid search on more parameters than just maximum depth.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.tree import DecisionTreeClassifier, export_text

## Data Importing

In [ ]:
X = pd.read_csv('/home/charliecryer/Documents/Machine_Learning/DataSets/Heart-Disease-X.csv')
y = pd.read_csv('/home/charliecryer/Documents/Machine_Learning/DataSets/Heart-Disease-y.csv')
print(X.shape)
X.head(3)

(66489, 209)


,age,height,weight,ap_hi,ap_lo,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,...,alco_0^2,alco_0 alco_1,alco_0 active_0,alco_0 active_1,alco_1^2,alco_1 active_0,alco_1 active_1,active_0^2,active_0 active_1,active_1^2
0,18393.0,168.0,62.0,110.0,80.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,20228.0,156.0,85.0,140.0,90.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,18857.0,165.0,64.0,130.0,70.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
print(y.shape)
y.head(3)

(66489, 1)


,cardio
0,0
1,1
2,1


Each dataset loaded properly, and the shapes of each dataset match. Let's look at the range of the dataset to make sure it is clean.

In [ ]:
X.describe()

,age,height,weight,ap_hi,ap_lo,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,...,alco_0^2,alco_0 alco_1,alco_0 active_0,alco_0 active_1,alco_1^2,alco_1 active_0,alco_1 active_1,active_0^2,active_0 active_1,active_1^2
count,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,...,66489.000000,66489.0,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.0,66489.000000
mean,19463.983275,164.312954,72.997406,126.384124,81.166012,0.654680,0.345320,0.754095,0.133917,0.111988,...,0.947615,0.0,0.187866,0.759750,0.052385,0.007896,0.044489,0.195762,0.0,0.804238
std,2468.911974,7.553720,12.382733,16.502193,9.386770,0.475476,0.475476,0.430626,0.340566,0.315355,...,0.222803,0.0,0.390608,0.427239,0.222803,0.088509,0.206179,0.396789,0.0,0.396789
min,10798.000000,143.000000,40.000000,60.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,17657.000000,159.000000,65.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
50%,19702.000000,165.000000,71.000000,120.000000,80.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
75%,21323.000000,170.000000,81.000000,140.000000,90.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
max,23713.000000,186.000000,107.000000,240.000000,182.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


Last time, I discovered some really high values for systolic blood pressure (ap_hi) in the cleaned dataset, which I removed by arbitrarily cutting out all systolic blood pressures greater than 350. Then, the highest value was 309.Now, we see the highest value is 240. Upon review of the data and the data cleaning, I realize that the 309 entry had a 0 diastolic pressure (ap_lo), which is impossible in a living human. I noticed that the data cleaning catches this problem, arbitrarily removing ap_lo and ap_hi values that are below 20.
So, the bottom line is all the data appears to be in acceptable ranges.

## Baseline Error Rate

In [ ]:
y.cardio.value_counts()

0    33927
1    32562
Name: cardio, dtype: int64

For baseline, we guess no for every input, since it is the majority of the dataset.

In [ ]:
print("Baseline error rate is: ", 1-33927/(33927+32562))

Baseline error rate is:  0.4897351441591842


## Feature Selection

It's important to bear in mind here that Decision Trees already do their own feature selection, so it's questionable whether or not feature selection will actually help. Sklearn says that feature selection can help. So, I'm going to perform feature selection, and compare decision trees trained with and without feature selection.

In [ ]:
def SelectFeatures(model, candidates, X, y):
        N = len(candidates)
        score = np.zeros(N)
        for k in range(N):
            model_k = model.copy()
            model_k.append(candidates[k])
            cv_results = cross_validate(decision_tree, X[model_k], y, n_jobs = -1)
            score[k] = cv_results['test_score'].mean()
        score_max = score.max()
        selected = candidates[score.argmax()]
        return (selected, score_max)

model = []
candidates = list(X.columns)
validation_score = []
decision_tree = DecisionTreeClassifier()


while (len(candidates) > 0):
    print('Candidates left: ', len(candidates), end='   ')
    (selected, score_max) = SelectFeatures(model, candidates, X, y)
    candidates.remove(selected)
    model.append(selected)
    validation_score.append(score_max)
    print('selected: ', selected, score_max)

results = pd.DataFrame()
results['feature added'] = model
results['validation score'] = validation_score

ix = results['validation score'].idxmax()
print()
print('use features 0 to ', ix, end='       ')
score_max = results['validation score'].max()
print('max validation score = ', round(score_max, 3))
model_opt = model[0:ix+1]

Candidates left:  209   selected:  ap_hi 0.7130802914856806
Candidates left:  208   selected:  cholesterol_3 0.724450589730983
Candidates left:  207   selected:  smoke_0 alco_0 0.7260147686473314
Candidates left:  206   selected:  gender_1 gluc_1 0.726465967709067
Candidates left:  205   selected:  gluc_3 active_1 0.7265261214761798
Candidates left:  204   selected:  gender_1 gender_2 0.7265411613317971
Candidates left:  203   selected:  cholesterol_3^2 0.7265411613317971
Candidates left:  202   selected:  ap_hi^2 0.7265411613317971
Candidates left:  201   selected:  cholesterol_1 cholesterol_2 0.7265411613317971
Candidates left:  200   selected:  cholesterol_1 cholesterol_3 0.7265411613317971
Candidates left:  199   selected:  gluc_1 gluc_2 0.7265411613317971
Candidates left:  198   selected:  cholesterol_2 cholesterol_3 0.7265411613317971
Candidates left:  197   selected:  gluc_2 gluc_3 0.7265411613317971
Candidates left:  196   selected:  alco_0 alco_1 0.7265411613317971
Candidates 

/home/charliecryer/anaconda3/envs/ml/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

After a few hours of runtime, then the error, I decided to cut off the feature selection, as, it seemed like the new features added marginal benefit or harmed the predictions.

In [ ]:
results = pd.DataFrame()
results['feature added'] = model
results['validation score'] = validation_score

ix = results['validation score'].idxmax()
print()
print('use features 0 to ', ix, end='       ')
score_max = results['validation score'].max()
print('lowest validation error = ', round(1-score_max, 3))
model_opt = model[0:ix+1]


use features 0 to  22       lowest validation error =  0.273


It looks like 22 features are the most important. The validation error does not beat the last one.

In [ ]:
model_opt

['ap_hi',
 'cholesterol_3',
 'smoke_0 alco_0',
 'gender_1 gluc_1',
 'gluc_3 active_1',
 'gender_1 gender_2',
 'cholesterol_3^2',
 'ap_hi^2',
 'cholesterol_1 cholesterol_2',
 'cholesterol_1 cholesterol_3',
 'gluc_1 gluc_2',
 'cholesterol_2 cholesterol_3',
 'gluc_2 gluc_3',
 'alco_0 alco_1',
 'gluc_1 gluc_3',
 'smoke_0 smoke_1',
 'active_0 active_1',
 'ap_hi cholesterol_3',
 'cholesterol_3 gluc_2',
 'gluc_2 smoke_0',
 'cholesterol_2 active_0',
 'cholesterol_2 smoke_1',
 'cholesterol_2 gluc_3']

## Hyperparameter Optimization with Grid Search
My kernel crashed and I had to reset it. I don't have time to repeat the feature selection, so I just copied the last results over to the restarted kernel.

In [ ]:
model_opt = ['ap_hi',
 'cholesterol_3',
 'smoke_0 alco_0',
 'gender_1 gluc_1',
 'gluc_3 active_1',
 'gender_1 gender_2',
 'cholesterol_3^2',
 'ap_hi^2',
 'cholesterol_1 cholesterol_2',
 'cholesterol_1 cholesterol_3',
 'gluc_1 gluc_2',
 'cholesterol_2 cholesterol_3',
 'gluc_2 gluc_3',
 'alco_0 alco_1',
 'gluc_1 gluc_3',
 'smoke_0 smoke_1',
 'active_0 active_1',
 'ap_hi cholesterol_3',
 'cholesterol_3 gluc_2',
 'gluc_2 smoke_0',
 'cholesterol_2 active_0',
 'cholesterol_2 smoke_1',
 'cholesterol_2 gluc_3']

X = pd.read_csv('/home/charliecryer/Documents/Machine_Learning/DataSets/Heart-Disease-X.csv')
y = pd.read_csv('/home/charliecryer/Documents/Machine_Learning/DataSets/Heart-Disease-y.csv')

grid = {'max_depth':np.arange(1,15),
        'min_samples_leaf':np.arange(1,7),
        'min_samples_split':np.arange(1,7)}

dtree = DecisionTreeClassifier()
grid_search_dt = GridSearchCV(dtree, param_grid=grid, return_train_score=True)

In [ ]:
Xs = X[model_opt]

grid_search_dt.fit(Xs,y)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'min_samples_leaf': array([1, 2, 3, 4, 5, 6]),
                         'min_samples_split': array([1, 2, 3, 4, 5, 6])},
             return_train_score=True)

In [ ]:
pd.concat([pd.DataFrame(grid_search_dt.cv_results_['params']), pd.DataFrame(grid_search_dt.cv_results_['mean_test_score'])], axis=1)

,max_depth,min_samples_leaf,min_samples_split,0
0,1,1,1,0.713531
1,1,1,2,0.713531
2,1,1,3,0.713531
3,1,1,4,0.713531
4,1,1,5,0.713531
...,...,...,...,...
499,14,6,2,0.727338
500,14,6,3,0.727338
501,14,6,4,0.727338
502,14,6,5,0.727338


In [ ]:
print('Best parameters:', grid_search_dt.best_params_)
print('Best test error:', 1-grid_search_dt.best_score_)


Best parameters: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 1}
Best test error: 0.2724812378734308


In [ ]:
print('test error:', 1-grid_search_dt.best_estimator_.score(Xs,y))

test error: 0.27159379747025825


In [ ]:


odtree_string = export_text(grid_search_dt.best_estimator_,
                          feature_names=list(Xs.columns),
                          decimals=2,
                          spacing=5)
print(odtree_string)

|----- ap_hi^2 <= 16770.50
|     |----- ap_hi cholesterol_3 <= 97.50
|     |     |----- ap_hi <= 118.50
|     |     |     |----- cholesterol_2 active_0 <= 0.50
|     |     |     |     |----- ap_hi^2 <= 10712.50
|     |     |     |     |     |----- gender_1 gluc_1 <= 0.50
|     |     |     |     |     |     |----- gluc_2 smoke_0 <= 0.50
|     |     |     |     |     |     |     |----- ap_hi^2 <= 7662.50
|     |     |     |     |     |     |     |     |----- class: 0
|     |     |     |     |     |     |     |----- ap_hi^2 >  7662.50
|     |     |     |     |     |     |     |     |----- cholesterol_2 smoke_1 <= 0.50
|     |     |     |     |     |     |     |     |     |----- smoke_0 alco_0 <= 0.50
|     |     |     |     |     |     |     |     |     |     |----- class: 0
|     |     |     |     |     |     |     |     |     |----- smoke_0 alco_0 >  0.50
|     |     |     |     |     |     |     |     |     |     |----- class: 0
|     |     |     |     |     |     |     |     |----- ch

### Looks like the new feature selection didn't improve on the last iterations test score of 0.265

In [ ]:
grid_search_dt2 = GridSearchCV(dtree, param_grid=grid, return_train_score=True)
grid_search_dt2.fit(X,y)

KeyboardInterrupt: 

In [ ]:
pd.concat([pd.DataFrame(grid_search_dt.cv_results_['params']), pd.DataFrame(grid_search_dt.cv_results_['mean_test_score'])], axis=1)